Завантажує кольорові зображення з raw/color/ і для кожного генерує:

1.Відтінки сірого (grayscale/)

2.Сегментацію (Otsu-бінаризація + контури → segmented/)


In [ ]:
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm

RAW_ROOT = Path("data/plantvillage/raw")
COLOR_DIR = RAW_ROOT / "color"
GRAY_DIR  = RAW_ROOT / "grayscale"
SEG_DIR   = RAW_ROOT / "segmented"

def generate_gray_and_segmented():
    GRAY_DIR.mkdir(parents=True, exist_ok=True)
    SEG_DIR.mkdir(parents=True, exist_ok=True)

    for img_path in tqdm(list(COLOR_DIR.glob("*.*")), desc="Generating gray/segmented"):
        img_color = cv2.imread(str(img_path))
        if img_color is None:
            continue

        # 1) Grayscale
        img_gray = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)
        cv2.imwrite(str(GRAY_DIR / img_path.name), img_gray)

        # 2) Segmented: Otsu + контури
        _, thresh = cv2.threshold(img_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        seg_img = np.zeros_like(img_color)
        cv2.drawContours(seg_img, contours, -1, (0,255,0), 2)
        cv2.imwrite(str(SEG_DIR / img_path.name), seg_img)

if __name__ == "__main__":
    generate_gray_and_segmented()
    print("✅ Gray and segmented images generated.")
